# Intelligent Agents: Reflex-Based Agents for the Vacuum-cleaner World

Student Name: Ryan Sweeney

I have used the following AI tools: pytorch

I understand that my submission needs to be my own work: RS

## Instructions

Total Points: Undergrads 100 + 5 bonus / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a HTML file.

In Visual Studio Code: Use Export (click on ... in the menu bar) to save your notebook as a HTML file.

In Colab you need to save the notebook on GoogleDrive and then add the following block
```
!pip install nbconvert

from google.colab import drive
drive.mount('/content/drive')
!jupyter nbconvert --to html /content/drive/MyDrive/Colab\ Notebooks/Copy\ of\ robot_vacuum.ipynb
```

You will need to fix the file location on your GoogleDrive.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex-based agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square in the room has been cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can clean the current square (action `suck`) or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [1]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty, n):
    return np.random.choice(actions)

In [2]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty, 5)

'north'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall repeatedly or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simple simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns a performance measure which is here the number of cleaned squares (since the room is infinite and all squares are constantly dirty, the agent can never clean the whole room as required in the PEAS description above). The energy budget of the agent is specified as `max_steps`.

In [3]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0

    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty, 5)
        if (verbose): print("step", i , "- action:", action)

        if (action == "suck"):
            num_cleaned = num_cleaned + 1

    return num_cleaned



Do one simulation run with a simple randomized agent that has enough energy for 20 steps.

In [4]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: south
step 1 - action: east
step 2 - action: suck
step 3 - action: suck
step 4 - action: suck
step 5 - action: north
step 6 - action: north
step 7 - action: east
step 8 - action: south
step 9 - action: suck
step 10 - action: north
step 11 - action: north
step 12 - action: north
step 13 - action: north
step 14 - action: suck
step 15 - action: west
step 16 - action: north
step 17 - action: east
step 18 - action: south
step 19 - action: west


5

# Tasks

## General [10 Points]

1. Make sure that you use the latest version of this notebook. Sync your forked repository and pull the latest revision.
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement intelligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formatted professionally.
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your PDF file is readable. For example, long lines are cut off in the PDF file. You don't have control over page breaks, so do not worry about these.
4. Document your code. Add a short discussion of how your implementation works and your design choices.


## Task 1: Implement a simulation environment [20 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty. ([Help with random numbers and arrays in Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/random_numbers_and_arrays.ipynb))
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment should be a function like the `simple_environment()` and needs to work with the simple randomized agent program from above. **Use the same environment for all your agent implementations in the tasks below.**

*Note on debugging:* Debugging is difficult. Make sure your environment prints enough information when you use `verbose = True`. Also, implementing a function that the environment can use to displays the room with dirt and the current position of the robot at every step is very useful.  

In [5]:
# Your code and description goes here
import numpy as np
import random

def create_environment(n, p):
    environment = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if random.random() < p:
                environment[i][j] = 1
    return environment

def print_environment(environment, position):
    n = len(environment)
    for i in range(n):
        for j in range(n):
            if (i, j) == position:
                print("X", end = " ")
            elif environment[i][j] == 1:
                print("D", end = " ")
            else:
                print("C", end = " ")
        print()

def simple_environment(agent, n, p, verbose = False):
    environment = create_environment(n, p)
    num_cleaned = 0
    steps = 0  # Initialize steps counter
    position = (random.randint(0, n-1), random.randint(0, n-1))
    total_dirty = np.sum(environment)  # Count total dirty cells initially
    
    if (verbose): print_environment(environment, position)
    
    while num_cleaned < total_dirty:  # Continue until all dirty cells are cleaned
        dirty = environment[position[0]][position[1]] == 1
        bumpers = {"north" : position[0] == 0, "south" : position[0] == n-1, "west" : position[1] == 0, "east" : position[1] == n-1}

        action = agent(bumpers, dirty, n)
        if (verbose): print("step", steps , "- action:", action)

        if (action == "suck"):
            if dirty:  # Only increment num_cleaned if the cell was dirty
                environment[position[0]][position[1]] = 0
                num_cleaned += 1
        else:
            if action == "north" and not bumpers["north"]:
                position = (position[0]-1, position[1])
            elif action == "south" and not bumpers["south"]:
                position = (position[0]+1, position[1])
            elif action == "west" and not bumpers["west"]:
                position = (position[0], position[1]-1)
            elif action == "east" and not bumpers["east"]:
                position = (position[0], position[1]+1)
                
        steps += 1  # Increment steps for every action taken
    
    if (verbose): print_environment(environment, position)
    return steps  # Return the total number of steps taken as performance measure


In [6]:
# Test the environment
simple_environment(simple_randomized_agent, 5, 0.2, verbose=True)

C C C C D 
X C C C C 
C C C C C 
C C C C D 
D D D C D 
step 0 - action: east
step 1 - action: south
step 2 - action: south
step 3 - action: north
step 4 - action: suck
step 5 - action: west
step 6 - action: north
step 7 - action: north
step 8 - action: west
step 9 - action: north
step 10 - action: north
step 11 - action: suck
step 12 - action: north
step 13 - action: suck
step 14 - action: south
step 15 - action: suck
step 16 - action: north
step 17 - action: west
step 18 - action: east
step 19 - action: west
step 20 - action: suck
step 21 - action: east
step 22 - action: north
step 23 - action: north
step 24 - action: west
step 25 - action: north
step 26 - action: west
step 27 - action: west
step 28 - action: suck
step 29 - action: north
step 30 - action: north
step 31 - action: suck
step 32 - action: east
step 33 - action: west
step 34 - action: suck
step 35 - action: east
step 36 - action: north
step 37 - action: north
step 38 - action: south
step 39 - action: south
step 40 - action

492

## Task 2:  Implement a simple reflex agent [10 Points]

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ Agents cannot directly use variable in the environment. They only gets the percepts as the arguments to the agent function.

In [7]:
# Your code and description g#create a smarter agent
def simple_reflex_agent(bumpers, dirty, n):
    #note I insert n but do not use it so all agents are compatible with the environment
    if dirty:
        return "suck"
    else:
        # Randomly choose a direction that is not blocked
        while True:
            action = np.random.choice(["north", "east", "west", "south"])
            if not bumpers[action]:
                return action
    

In [8]:
simple_environment(simple_reflex_agent, 5, 0.2, verbose=True)

C C C C C 
X C C C D 
C C C C D 
C C C C C 
C D C C C 
step 0 - action: east
step 1 - action: north
step 2 - action: south
step 3 - action: north
step 4 - action: west
step 5 - action: south
step 6 - action: south
step 7 - action: north
step 8 - action: east
step 9 - action: east
step 10 - action: west
step 11 - action: south
step 12 - action: south
step 13 - action: north
step 14 - action: south
step 15 - action: north
step 16 - action: south
step 17 - action: south
step 18 - action: suck
step 19 - action: north
step 20 - action: north
step 21 - action: north
step 22 - action: north
step 23 - action: east
step 24 - action: south
step 25 - action: south
step 26 - action: east
step 27 - action: north
step 28 - action: east
step 29 - action: suck
step 30 - action: west
step 31 - action: north
step 32 - action: south
step 33 - action: south
step 34 - action: east
step 35 - action: suck
C C C C C 
C C C C C 
C C C C X 
C C C C C 
C C C C C 


36

## Task 3: Implement a model-based reflex agent [20 Points]

Model-based agents use a state to keep track of what they have done and perceived so far. Your agent needs to find out where it is located and then keep track of its current location. You also need a set of rules based on the state and the percepts to make sure that the agent will clean the whole room. For example, the agent can move to a corner to determine its location and then it can navigate through the whole room and clean dirty squares.

Describe how you define the __agent state__ and how your agent works before implementing it. ([Help with implementing state information on Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/store_agent_state_information.ipynb))

In [9]:
# Your short description of the state and your implementation goes here
#My implementation of the model based agent uses a spiral pattern to hit all the squares, it starts in a wall finding mode, then it spirals inwards to clean the room. It keeps track of the number of steps it has taken in the current direction and the number of turns it has made since the last decrease in steps allowed. I define the agent state as the visited squares and the current position of the agent. it also tracks steps since last turn and turns made since last decrease in steps allowed. My agent works by using these to follow the spiral pattern i mentioned earlier.

In [10]:
class SpiralerAgent:
    def __init__(self, n):
        self.n = n  # Environment size
        self.position = (random.randint(0, n-1), random.randint(0, n-1))  # Random start position
        self.orientation = 'North'
        self.visited = set()  # Keep track of visited cells
        self.mode = 'Find Wall'
        self.steps_since_last_turn = 0  # Steps taken in the current direction
        self.steps_allowed = n - 1  # Max steps allowed in the current spiral arm
        self.turns_made = 0  # Turns made since the last decrease in steps_allowed

    def update_orientation_for_spiral(self):
        # Adjust the agent's orientation for spiraling: turn right
        orientation_order = ['North', 'East', 'South', 'West']
        current_index = orientation_order.index(self.orientation)
        self.orientation = orientation_order[(current_index + 1) % 4]  # Turn right
        self.turns_made += 1
        # Every 2 turns, reduce the steps allowed in the current direction to tighten the spiral
        if self.turns_made % 3 == 0:
            self.steps_allowed -= 1
        self.steps_since_last_turn = 0  # Reset counter after each turn

    def can_move(self, bumpers):
        # Check if the agent can move forward in its current orientation without hitting a wall
        return not bumpers[self.orientation.lower()] and self.steps_since_last_turn < self.steps_allowed
    
    def decide_action(self, bumpers, dirty, n):
        if dirty:
            return "suck"
        
        if self.mode == 'Find Wall':
            if any(bumpers.values()):
                self.mode = 'Spiral Inwards'
                self.update_orientation_for_spiral()
            else:
                return self.orientation.lower()
        
        if self.mode == 'Spiral Inwards':
            if self.can_move(bumpers):
                self.steps_since_last_turn += 1  # Increment steps taken in the current direction
                return self.orientation.lower()
            else:
                self.update_orientation_for_spiral()  # Turn right when facing a wall or reaching steps limit
                return self.orientation.lower()
    
        # Default action if none above apply
        return "north"

    def __call__(self, bumpers, dirty, n):
        return self.decide_action(bumpers, dirty, n)


In [11]:
# Instantiate the spiraler agent
n = 5  # Environment size
spiraler_agent = SpiralerAgent(n)

# Run the environment with the spiraler agent
max_steps = 100  # Define maximum steps to simulate
num_cleaned = simple_environment(spiraler_agent, n, 0.2, verbose=True)

print(f"Number of steps: {num_cleaned}")
     

C C D C C 
C D D C C 
C C C D C 
C C C C C 
X C C C D 
step 0 - action: east
step 1 - action: east
step 2 - action: east
step 3 - action: east
step 4 - action: suck
step 5 - action: south
step 6 - action: west
step 7 - action: west
step 8 - action: west
step 9 - action: west
step 10 - action: north
step 11 - action: north
step 12 - action: north
step 13 - action: north
step 14 - action: east
step 15 - action: east
step 16 - action: suck
step 17 - action: east
step 18 - action: east
step 19 - action: south
step 20 - action: south
step 21 - action: south
step 22 - action: west
step 23 - action: west
step 24 - action: west
step 25 - action: north
step 26 - action: north
step 27 - action: suck
step 28 - action: north
step 29 - action: east
step 30 - action: east
step 31 - action: south
step 32 - action: south
step 33 - action: suck
step 34 - action: west
step 35 - action: west
step 36 - action: north
step 37 - action: east
step 38 - action: suck
C C C C C 
C C X C C 
C C C C C 
C C C C C 


## Task 4: Simulation study [30 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results using tables and graphs. Discuss the differences between the agents.
([Help with charts and tables in Python](https://github.com/mhahsler/CS7320-AI/blob/master/HOWTOs/charts_and_tables.ipynb))

In [12]:
# Your code goes here
#5x5
n = 5  # Example for 5x5 environment, repeat for 10x10 and 100x100
p = 0.2

Threads = []
# Run the simulations
randomized_agent_steps = [simple_environment(simple_randomized_agent, n, p, verbose=False) for _ in range(100)]
simple_reflex_agent_steps = [simple_environment(simple_reflex_agent, n, p, verbose=False) for _ in range(100)]
model_based_reflex_agent_steps = [simple_environment(SpiralerAgent(n), n, p, verbose=False) for _ in range(100)]

# Output the average steps taken to clean the entire environment
print(f"{n}x{n}")
print("Randomized Agent: Average Steps =", np.mean(randomized_agent_steps))
print("Simple Reflex Agent: Average Steps =", np.mean(simple_reflex_agent_steps))
print("Model-based Reflex Agent: Average Steps =", np.mean(model_based_reflex_agent_steps))

#note since jupyter notebooks are not good with multithreading I did not run the 100x100 I wrote seperate files with the necessary functions and multithreading support so I could run the entire scene in a couple minutes instead of a very long time.


5x5
Randomized Agent: Average Steps = 415.36
Simple Reflex Agent: Average Steps = 102.16
Model-based Reflex Agent: Average Steps = 29.98


Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | 429.19| 91.86| 30.58|
| 10x10   | 3205.79| 968.39| 160.76|
| 100x100 | 824978.03| 341183.86| 17029.91|

Add charts to compare the performance of the different agents.

5x5
![alt-text](5x5.jpeg "5x5")
10x10.jpeg
![alt-text](10x10.jpeg "10x10")
100x100.jpeg
![alt-text](100x100.jpeg "100x100")


In [13]:
# Your graphs and discussion of the results goes here
5x5.jpeg


SyntaxError: invalid syntax (1508700418.py, line 2)

## Task 5: Robustness of the agent implementations [10 Points]

Describe how **your agent implementations** will perform

* if it is put into a rectangular room with unknown size,
* if the cleaning area can have an irregular shape (e.g., a hallway connecting two rooms), or
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).

In [ ]:
# Answer goes here
#Rectangular Room:
#in the rectangular room example the model based agent would break because it was designed to sporal inwards in a square, so the count of the edge is very important. If it were more of a lawnmower pattern it would be fine, but that is not what I coded. The other two agents would perform similiarly to how they do with the square room because they move randomly.

## Advanced task: Obstacles

* __Graduate students__ need to complete this task [10 points]
* __Undergraduate students__ can attempt this as a bonus task [max +5 bonus points].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can.

In [ ]:
# Your code and discussion goes here
import numpy as np
import random

def create_environment(n, p_dirty, p_obstacle):
    environment = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if random.random() < p_dirty:
                environment[i][j] = 1  # Dirty cell
            elif random.random() < p_obstacle:
                environment[i][j] = 2  # Obstacle
    return environment

def print_environment(environment, position):
    n = len(environment)
    for i in range(n):
        for j in range(n):
            if (i, j) == position:
                print("X", end=" ")
            elif environment[i][j] == 1:
                print("D", end=" ")  # Dirty
            elif environment[i][j] == 2:
                print("O", end=" ")  # Obstacle
            else:
                print("C", end=" ")  # Clean
        print()

def simple_environment(agent, n, p_dirty, p_obstacle, verbose=False):
    environment = create_environment(n, p_dirty, p_obstacle)
    num_cleaned = 0
    steps = 0
    position = (random.randint(0, n-1), random.randint(0, n-1))
    total_dirty = np.sum(environment == 1)  # Count total dirty cells initially
    
    if (verbose): print_environment(environment, position)
    
    while num_cleaned < total_dirty:
        dirty = environment[position[0]][position[1]] == 1
        obstacle = environment[position[0]][position[1]] == 2
        bumpers = {
            "north": position[0] == 0 or environment[position[0]-1][position[1]] == 2,
            "south": position[0] == n-1 or environment[position[0]+1][position[1]] == 2,
            "west": position[1] == 0 or environment[position[0]][position[1]-1] == 2,
            "east": position[1] == n-1 or environment[position[0]][position[1]+1] == 2
        }

        action = agent(bumpers, dirty, n)
        if (verbose): print("step", steps , "- action:", action)

        if action == "suck" and dirty:
            environment[position[0]][position[1]] = 0
            num_cleaned += 1
        elif action in ["north", "south", "west", "east"] and not obstacle:
            # Movement logic remains the same, but now bumpers also consider obstacles
            if action == "north" and not bumpers["north"]:
                position = (position[0]-1, position[1])
            elif action == "south" and not bumpers["south"]:
                position = (position[0]+1, position[1])
            elif action == "west" and not bumpers["west"]:
                position = (position[0], position[1]-1)
            elif action == "east" and not bumpers["east"]:
                position = (position[0], position[1]+1)
                
        steps += 1
    
    if (verbose): print_environment(environment, position)
    return steps


This environment is interesting because it actually fully breaks my model based agent, the model agent does not know what to do when it hits an obsticle and gets stuck in a loop. The other agents are not affected as much, but they do take slightly longer to clean the room. Some special logic would need to be added to the spiral agent to make it first find a corner, then it can know the count of how far over it is from the wall and determine if it is an obsticle or a wall it hits, then it would need to employ some sort of following tactic to follow the edge of the obticle until it reaches the other side. This would be a very complex task to implement and would require a lot of extra code.

## More advanced implementation tasks

* __Agent for and environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square (note that this is actually depth-first search).

* __Utility-based agent:__ Change the environment for a $5 \times 5$ room, so each square has a fixed probability of getting dirty again. For the implementation, we give the environment a 2-dimensional array of probabilities. The utility of a state is defined as the number of currently clean squares in the room. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 100000 time steps. To do this, the agent needs to learn the probabilities with which different squares get dirty again. This is very tricky!

In [ ]:
# Your ideas/code
For the first task you could pretty easily have the agent store the states of visited squares in cordinates from its starting point, then it could move to the closest unvisited square, however it would be tough to do this and account for objects simply because it does not have a way to distinguish an object from a wall, so it might forget squares exist if it assumes an object is the wall, or it might think squares still exist past the barricade if it assumes it is an object an not a wall. Because of this, I would think it might be better to do some sort of width test and height test to start then once it knows the size of the room, it can do DFS on the squares.
    
    For the second task I would think you need to somehow keep track of which squares it has cleaned over number of moves since last visited that square and then it would need to estimate the probabilities and build a path that visits the higher probability squares too. I dont know what the exact math would be but it would have to be some weighted probability of moves since last seen. Maybe we could first do a training phase where the path is constant and then after is guesses the probability from the moves in the training phase it optimizes the path to hit the most dirty squares probabilistically.